### Set up Workspace

In [1]:
%matplotlib inline

import xarray as xr
import intake
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cftime
import xesmf as xe
import pickle

# util.py is in the local directory
# it contains code that is common across project notebooks
# or routines that are too extensive and might otherwise clutter
# the notebook design
import util 
output_path='/home/jovyan/cmip6-outreach-tool/output/'

/srv/conda/envs/notebook/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['stac-catalog', 'stac-collection', 'stac-item'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


### Run Scripts to Create Raw Dataset Dictionary and Time Series Dataset

In [2]:
import CreateTimeSeries
dict_timeSeries = CreateTimeSeries.dict_timeSeries
this_experiment_id = CreateTimeSeries.this_experiment_id
this_variable_id = CreateTimeSeries.this_variable_id
this_table_id = CreateTimeSeries.this_table_id
this_grid_label = CreateTimeSeries.this_grid_label

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 60 group(s)
---------------historical---------------
CMIP.MIROC.MIROC6.historical.Amon.gn
Overwrite existing file: nearest_s2d_128x256_160x320.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable tas as the horizontal dimensions for this dataset.
CMIP.MRI.MRI-ESM2-0.historical.Amon.gn
Overwrite existing file: nearest_s2d_160x320_160x320.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable tas as the horizontal dimensions for this dataset.
CMIP.NASA-GISS.GISS-E2-1-H.historical.Amon.gn
Overwrite existing file: nearest_s2d_90x144_160x320.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable tas as the horizontal dimensions for this dataset.
CMIP.MOH

### This Finds the Closest Lat/Lon in the Grid to the Lat/Lon you want

In [6]:
def find_loc_inds(thislat,thislon):
    lats = dict_timeSeries['historical']['lat'].values
    lons = dict_timeSeries['historical']['lon'].values
    lat_ind = np.where(abs(lats-thislat) == np.amin(abs(lats-thislat)))[0][0]
    lon_ind = np.where(abs(lons-thislon) == np.amin(abs(lons-thislon)))[0][0]
    return lat_ind, lon_ind
lat_ind, lon_ind = find_loc_inds(thislat = 47.6062,thislon = 237.67)

#warm point: (thislat = 4,thislon = 140) #

### This is how you get an xarray for a given scenario

In [5]:
dict_timeSeries['historical']
# Get indices for specific lat and lon

<xarray.Dataset>
Dimensions:          (lat: 160, lon: 320, time: 1980)
Coordinates:
  * time             (time) object 1850-01-15 00:00:00 ... 2014-12-15 00:00:00
  * lon              (lon) float64 0.0 1.125 2.25 3.375 ... 356.6 357.8 358.9
  * lat              (lat) float64 -89.14 -88.03 -86.91 ... 86.91 88.03 89.14
Data variables:
    CAMS-CSM1-0      (time, lat, lon) float64 dask.array<chunksize=(488, 160, 320), meta=np.ndarray>
    MIROC6           (time, lat, lon) float64 dask.array<chunksize=(1980, 160, 320), meta=np.ndarray>
    MRI-ESM2-0       (time, lat, lon) float64 dask.array<chunksize=(1980, 160, 320), meta=np.ndarray>
    GISS-E2-1-H      (time, lat, lon) float64 dask.array<chunksize=(600, 160, 320), meta=np.ndarray>
    UKESM1-0-LL      (time, lat, lon) float64 dask.array<chunksize=(600, 160, 320), meta=np.ndarray>
    SAM0-UNICON      (time, lat, lon) float64 dask.array<chunksize=(600, 160, 320), meta=np.ndarray>
    NorCPM1          (time, lat, lon) float64 dask.array<

### Make figure for given Lat Lon

In [ ]:
normalizeHistorical = True

fig = plt.figure(figsize=(10,6))
lstyle = ['-',':','--','-.','-']
fname='Historical-and-ScenarioTimeSeries'
for i in range(0,2):#len(this_experiment_id)):
    experiment_id = this_experiment_id[i]
    # Read in data for experiment id
    print('---------------'+experiment_id+'---------------')
    modelnames_toplot = dict_timeSeries[experiment_id+'_modelnameToPlot']
    ds = dict_timeSeries[experiment_id]
    
    # Calculate annual mean
    ds_annual=ds.groupby('time.year').mean('time')
    ds_annual=ds_annual.isel(lat=lat_ind,lon=lon_ind)
    
    if normalizeHistorical:
        # Normalize to historical
        if experiment_id=='historical':
            baseClim = ds_annual.isel(year=range(0,50)).mean(dim='year')
        ds_annual=ds_annual-baseClim
        fname=fname+'_normalized'
    
    for modelname in modelnames_toplot:
        print(modelname)
        plt.plot(ds_annual['year'],ds_annual[modelname],label=modelname,linestyle=lstyle[i])
plt.legend()
plt.xlabel('Year')
plt.xlim([1850,2200])
plt.ylabel('Temperature')
fig.savefig(output_path +fname+'.png', bbox_inches='tight') 

---------------historical---------------
